In [1]:
import open3d as o3d
import numpy as np
from tqdm.notebook import tqdm
import pptk

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def find_closest_points(pcd_A, pcd_B):
    # Build KNN search tree
    search_tree = o3d.geometry.KDTreeFlann(pcd_A)

    # Find closest points
    closest_points = []
    for point in tqdm(pcd_B.points):
        _, idx, _ = search_tree.search_knn_vector_3d(point, 1)
        closest_point = pcd_A.points[idx[0]]
        closest_points.append(closest_point)

    return np.array(closest_points)

def find_closest_points_idxs(pcd_A, pcd_B):
    search_tree = o3d.geometry.KDTreeFlann(pcd_A)

    closest_points_indices = []
    for point in tqdm(pcd_B.points):
        _, idx, _ = search_tree.search_knn_vector_3d(point, 1)
        closest_points_indices.append(idx[0])

    return closest_points_indices

In [3]:
def calculate_projection_accuracy(pcd_A, pcd_B, closest_points_indices, labels_A, labels_B):
    correct_labels = 0
    total_points = len(pcd_B.points)

    for i, idx in tqdm(enumerate(closest_points_indices), total=len(closest_points_indices)):
        if labels_A[idx] == labels_B[i]:
            correct_labels += 1

    accuracy = correct_labels / total_points
    return accuracy

In [4]:
# Load point clouds from file
pcd_A_np = np.load("/home/luc/PycharmProjects/Pointnet_Pointnet2_pytorch/data/PatrickData/Piazza/Piazza.npy")
pcd_B_np = np.load("/home/luc/PycharmProjects/Pointnet_Pointnet2_pytorch/data/PatrickData/Piazza/piazza.npy_full.npy")
labels_A = np.copy(pcd_A_np[:,-1])
labels_B = np.copy(pcd_B_np[:,-1])

In [7]:
pcd_A = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(pcd_A_np[:,:3]))
pcd_B = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(pcd_B_np[:,:3]))

In [8]:
del pcd_A_np, pcd_B_np

In [9]:
# Find closest points indices
closest_points_indices = find_closest_points_idxs(pcd_A, pcd_B)

# Calculate label accuracy
accuracy = calculate_projection_accuracy(pcd_A, pcd_B, closest_points_indices, labels_A, labels_B)

print(f"Label accuracy when mapping from pcd_B to pcd_A: {accuracy * 100:.2f}%")

  0%|          | 0/55108783 [00:00<?, ?it/s]

  0%|          | 0/55108783 [00:00<?, ?it/s]

Label accuracy when mapping from pcd_B to pcd_A: 99.77%


In [10]:
def visualize_incorrect_points(pcd_A, pcd_B, closest_points_indices, labels_A, labels_B):
    incorrect_points = []

    for i, idx in tqdm(enumerate(closest_points_indices), total=len(closest_points_indices)):
        if labels_A[idx] != labels_B[i]:
            incorrect_points.append(pcd_B.points[i])

    incorrect_points_pcd = o3d.geometry.PointCloud()
    incorrect_points_pcd.points = o3d.utility.Vector3dVector(incorrect_points)

    # Set the color for incorrect points, for example red (R: 1, G: 0, B: 0)
    incorrect_points_pcd.paint_uniform_color([1, 0, 0])

    # Visualize both pcd_B and incorrect points
    o3d.visualization.draw_geometries([pcd_A, incorrect_points_pcd], window_name='Incorrect Points Visualization')

# Visualize incorrect points
visualize_incorrect_points(pcd_A, pcd_B, closest_points_indices, labels_A, labels_B)

  0%|          | 0/55108783 [00:00<?, ?it/s]

In [11]:
labels_B/=255


In [14]:
max(labels_B)

0.0